In [452]:
import pandas as pd
import numpy as np
import random as rnd

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#머신러닝
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore')

In [453]:
#데이터 가져오기
train_df = pd.read_csv("/Users/seungwoo/Desktop/python/titanic/data_titanic/train.csv")
test_df = pd.read_csv("/Users/seungwoo/Desktop/python/titanic/data_titanic/test.csv")

In [454]:
#데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [455]:
#데이터 미리보기
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [456]:
#info를 통해 data 확인하기
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [457]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [458]:
#필요없는 데이터 생각하기 : PassengerId, Name, Ticket로 알 수 있는 데이터는 없을 것이다. 지우자. (passanger ID는 test에서 필요하므로 train에서만 삭제)
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)

In [459]:
test_df = test_df.drop(['Name', 'Ticket'], axis = 1)

In [460]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [461]:
#남은 데이터
train_df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked'],
      dtype='object')

데이터 하나하나 봐보기

In [462]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

Pclass 는 1등석, 2등석, 3등석이 있는데 3등석이 제일 많고, 2등석이 제일 적습니다.
 1, 2, 3 등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않습니다. 
 그렇기에 범주형(카테고리) 데이터로 인식하고 인코딩해야합니다.

In [463]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

In [464]:
train_df.drop(['Pclass'], axis = 1, inplace=True)
test_df.drop(['Pclass'], axis = 1, inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

이제 train과 test에는 정수 형태의 Pclass가 아닌 카테고리형의 Pclass가 존재합니다
이제 sex를 one-hot-encoding을 진행

In [465]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

Age는 연속형 데이터로 큰 처리가 필요없음, 하지만 일분 NaN 값 처리해줄 필요있음
NaN 값을 
1. 랜덤
2. 평균값
3. 중간값
4. 데이터버리기
택해야함

여기선 평균값으로 채워넣기


In [466]:
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)

SibSp & Panch

형재자매와, 부모님

Fare : 탑승료

Cabin의 결측 데이터는 약 600개, 이 데이터는 버리기로 결정

In [467]:
test_df["Fare"].fillna(0, inplace=True)

In [468]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Cabin     204 non-null    object 
 6   Embarked  889 non-null    object 
 7   1         891 non-null    uint8  
 8   2         891 non-null    uint8  
 9   3         891 non-null    uint8  
 10  Female    891 non-null    uint8  
 11  Male      891 non-null    uint8  
dtypes: float64(2), int64(3), object(2), uint8(5)
memory usage: 53.2+ KB


In [469]:
train_df = train_df.drop(['Cabin'], axis = 1)
test_df = test_df.drop(['Cabin'], axis = 1)

In [470]:
# Embarked  :탑승 항구를 의미
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [471]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [472]:
train_df['Embarked'].isnull().sum()

2

In [473]:
test_df['Embarked'].isnull().sum()

0

In [474]:
train_df['Embarked'].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)

In [475]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

In [476]:
test_df['Fare'].fillna(0, inplace=True)

In [477]:
train_df, test_df

(     Survived        Age  SibSp  Parch     Fare  1  2  3  Female  Male  S  C  \
 0           0  22.000000      1      0   7.2500  0  0  1       0     1  0  0   
 1           1  38.000000      1      0  71.2833  1  0  0       1     0  1  0   
 2           1  26.000000      0      0   7.9250  0  0  1       1     0  0  0   
 3           1  35.000000      1      0  53.1000  1  0  0       1     0  0  0   
 4           0  35.000000      0      0   8.0500  0  0  1       0     1  0  0   
 ..        ...        ...    ...    ...      ... .. .. ..     ...   ... .. ..   
 886         0  27.000000      0      0  13.0000  0  1  0       0     1  0  0   
 887         1  19.000000      0      0  30.0000  1  0  0       1     0  0  0   
 888         0  29.699118      1      2  23.4500  0  0  1       1     0  0  0   
 889         1  26.000000      0      0  30.0000  1  0  0       0     1  1  0   
 890         0  32.000000      0      0   7.7500  0  0  1       0     1  0  1   
 
      Q  
 0    1  
 1    

In [478]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [479]:
train_df.head()

,Survived,Age,SibSp,Parch,Fare,1,2,3,Female,Male,S,C,Q
0,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [480]:
test_df.head()

,PassengerId,Age,SibSp,Parch,Fare,1,2,3,Female,Male,S,C,Q
0,892,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
1,893,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
2,894,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
4,896,22.0,1,1,12.2875,0,0,1,1,0,0,0,1


In [481]:
features_drop = ['SibSp', 'Parch']

train_df = train_df.drop(features_drop, axis=1)
test_df = test_df.drop(features_drop, axis=1)

train_data = train_df.drop('Survived', axis=1)
target = train_df['Survived']
X_test  = test_df.drop("PassengerId",axis=1).copy()


train_data.shape, target.shape


((891, 10), (891,))

In [482]:
train_df.head()

,Survived,Age,Fare,1,2,3,Female,Male,S,C,Q
0,0,22.0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,71.2833,1,0,0,1,0,1,0,0
2,1,26.0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,8.0500,0,0,1,0,1,0,0,1


In [483]:
train_data.columns = train_data.columns.astype(str)
X_test.columns = X_test.columns.astype(str)


In [484]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [485]:
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [486]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [487]:
# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [488]:
clf = KNeighborsClassifier(n_neighbors = 13) # 모델 생성
scoring = 'accuracy'
# cross_validate(모델 객체, 훈련 인풋, 훈련 타겟)
score1 = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)

round(np.mean(score1)*100, 2)


71.94

In [489]:
clf = DecisionTreeClassifier()
score2 = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)

round(np.mean(score2)*100, 2)

77.89

In [490]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_data, target)
Y_pred = random_forest.predict(X_test)
random_forest.score(train_data, target)


0.9797979797979798

### 랜덤포레스트의 결과가 가장 좋음, 랜덤포레스트의 결과로 submission 파일 만들기

In [491]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

submission.to_csv('titanic.csv', index=False)